# Normalizing the Heliconia data set

## Problems with Heliconia data set

The data set hosted on Dryad is not **normalized**. The same variable appears in multiple columns, many rows refer to different observations (plants), and some categorical values (like population/year) have different labels in different parts of the table. This structure creates a few potential problems for analysis:

- **Updating is error-prone.** There are many opportunities for a single mistake to damage the data set's integrity. For example, suppose a newly trained researcher wanted to add a new observation, for a particular variety, population, and year. They need to interpret these values — and then interpret the data set structure — in order to find the correct data block in the correct sheet.  
- **Adding information changes the structure too broadly.** For example, adding just one new variable requires you to insert multiple columns in multiple worksheets. And if the researchers wanted to add data in subsequent years, they would need to add blocks of data to the right of existing blocks.
- **Potential queries are limited.** The current structure is biased in favor of a specific type of query. Specifically, it favors looking first at variety, then year, and then population. But what if you wanted to see all plants that had more than three bracts per inflorescence, and see if particular varieties or years are overrepresented?

These problems make the data structure difficult to interpret or update reliably. Importing the data into software will almost certainly fail.

We want to fix these problems by normalizing the data. A properly normalized data set (also called a “tidy” data set) organizes data in a consistent, efficient manner, with minimal redundancy. In a tidy data set, each variable is represented by a single column, and values for that variable appear only in that column.

## Structure of the raw data set

Open the [raw data set](https://drive.google.com/drive/folders/0B0B9tfvNpdIPfnY4UjNxVXNGaEladXI0Zk5vZS1WcDFDd3VLa0tiblROR0dZejhabHhLRjA). Use `File` > `Make a copy...` to save the file to your own account.

Show the sheet named `Bihai data`. Visually, there appear to be four “blocks” of data, each with five columns but variable number of rows. In each block, the five columns have names following this pattern:

- Population & Year
- plant ID
- Corolla length
- Mean bracts/inflorescence
- Seeds per plant

Within each block, each variable name is appended with the same code, e.g. “B08”, “FW08”, representing the population and year. The other sheets (`Caribaea red data` and `Caribaea yellow data`) are organized in a similar way, except they contain additional columns named `Morph`. 

## Eliminating unnecessary redundancies

View the sheet `Bihai data`. Within the first block, each variable is labeled according to population sampled and year (e.g., “Seeds per plant B08”). The researchers did this to  keep data in these columns associated with each other. But this labeling pattern could be misinterpreted. For example, a reader might think that the variable “Seeds per plant” is defined differently between populations and years. (Perhaps there is a special correction factor applied between populations.) (The variables are actually defined the same way for all populations, because the researchers wanted to compare populations.)

This labeling is unnecessary, because there is a separate variable (the first one) that records the population and year. To correct this problem we: (a) cut and paste the data for each population into the same five columns, and (b) rename the variables to remove the reference to population and year.

Be careful! Note that the third Bihai population is not labeled using the same pattern — the researchers called it “B09” instead of “Boeri Lake 2009”, which is potentially confusing to the reader. We fix this manually by copying and pasting “Boeri Lake 2008” from A2 into K2, changing the year, and then copying and pasting down the rest of this column. (The copy-and-paste is to avoid possible spelling errors.) Do the same for “FW09”.

## Representing species explicitly as a variable

The data are separated into three tabs according to species. This isn’t inherently confusing, but it could suggest that the three measurement variables are defined differently for each species. In addition, this arrangement could bias the way that you partition the data set. Finally, the separation impedes automation, as any processing would need to be repeated for each sheet. 

We can’t simply cut and paste into a single tab, because we would then lose information about the species of each plant. To solve this problem, we create a new variable to represent species: inserting a new column `A` to the left of `Population & Year`, name it `species`, and fill this column with the appropriate values. Repeat this change for the other two data sheets.

Switch back to the `Bihai data` sheet. The second two sets have a column for “Morph”, but the `Bihai` set doesn’t have this column. To ensure that columns line up properly when we merge the data, we insert a column to the left of column `C` and call it `morph`, but leave the values empty.

Cut and paste the data from `Caribaea red data` and `Caribaea yellow data` into `Bihai data`, directly below the existing data (row 139).

Rows 283-1000 are empty, so delete those rows  This sends a clear signal to future analysts that there are no data beyond row 282. (In Microsoft Excel, you cannot delete terminal rows, but you can hide them.) There should now be 7 columns (`A-G`) and 282 rows.

Rename this sheet to `Heliconia data`, since it contains the entire data set.

## Cleaning up the metadata

The data themselves are now organized in [first-normal form (1NF)](https://en.wikipedia.org/wiki/First_normal_form). Fundamentally this is all we need, but we can still tidy up the *metadata* that describes the variables. For examples, we can imagine that some of the column headers will generate problems for typical software that may interpret symbols (like ‘&’, ‘/‘, and ‘()’) in idiosyncratic (and therefore unexpected) ways. We therefore rename the column headers to make them simple while remaining descriptive. Python’s dictionary syntax can deal with spaces in the names, but other software might be less forgiving, so we replace spaces with Python-friendly underscores.

We also need to write descriptions of each variable. When we read the metadata (to figure out if we could stack the columns) we notice a lot of redundancy in defining each variable. By structuring the data the way they did, the researchers had, in essence, generated combinations of two variables: a measurement (e.g. corolla length) and a population. When we stacked the data as we did, we eliminated these redundancies and defined each measurement variable without reference to the population, thus keeping the variables independent.

Create a new sheet named `Heliconia meta`. Label the first 6 columns as such:

- A: `column`
- B: `variable`
- C: `original label`
- D: `type`
- E: `units`
- F: `description`

Fill in the values of of A-C appropriately. For `type`, classify the variable as categorical, integer, or continuous (i.e., `float`). For units, indicate the unit of measurement, if any. For the categorical variables, list the possible values defined by the experiment.

## Use a lookup table to represent a relationship

For one of the variables, `pop_and_year`, there are multiple possible values. Of course, the analyst receiving these data can just use aggregation or search methods to find out what they are. But we already have this information from the metadata, so why not include it? This would allow the analyst to check the validity of their methods against a list of known values. Create a new sheet, named `populations`. Label the first column `pop_and_year` and record each unique value of population/year. (There are 8 total.)

We could list these values in the units field of the variable descriptions. But a better idea is to create a separate sheet, named `populations`, containing each possible value on a separate line. This type of structure is called a **lookup table**. It allows another user to “look up” the population or year of a plant, given a `pop_and_year` label. 

This table allows you to associate one variable with two other variables that weren’t part of the original data set, but can be inferred. An analyst might find it useful later to aggregate data by year or by population, and having these associations recorded in the table would make their job much easier.

## Tidy data set

To finish up, delete the sheets that are no longer necessary:

- `Bihai meta`
- `Caribaea red data`
- `Caribaea red meta`
- `Caribaea yellow data`
- `Caribaea yellow meta`

For an example of how the data set should look after these changes, open [Heliconia tidy](https://docs.google.com/spreadsheets/d/1g1wE5-e_B3VQw6X2-yQTtw4NkUpc-bbk5x_lUBRkR5A/edit?pli=1#gid=1800680648).

## Importing data

One of the reasons for normalizing the data was to make it easier to import, so let's test that we can do that. Import and export methods are often caled "IO" (input-output) methods. `pandas` offers a few, including `read_csv()` and `read_excel()`. 

These methods can target files on your local file system, but they can also target files published on a website! You just need to know the exact URL of the desired file.

### Reading data from a public Google sheet

Google Drive can publish a Sheet as an xlsx file. For this to work from Python, make sure you have set sharing on the document to a public setting (either "Public on the web" or "Anyone with the link").\*

With the document open, examine the url of the sheet. It should be of the form:

> `'https://docs.google.com/spreadsheets/d/'` + *long string* + `'/edit?pli=1#gid='` + *short string*

The long string in the middle is called the **spreadsheet key**, and it uniquely identifies your document; the short string after `gid=` is the specific worksheet id. For example, in the URL (for `Heliconia tidy`) shown below,

- spreadsheet key is the string `'1g1wE5-e_B3VQw6X2-yQTtw4NkUpc-bbk5x_lUBRkR5A'`
- worksheet id  is the string `'1800680648'`

> `https://docs.google.com/spreadsheets/d/1g1wE5-e_B3VQw6X2-yQTtw4NkUpc-bbk5x_lUBRkR5A/edit?pli=1#gid=1800680648`

Copy the spreadsheet key. Then construct a new URL that points to an xlsx version of your spreadsheet. It contains a string that instructs Google to generate an up-to-date xlsx file and allow Python to retrieve it. See the example below.

\* *(It's possible to access your non-public documents from IPython Notebook if you import an authentication module, but this is much more complex and potentially insecure if you're not careful.)*

In [44]:
from pandas import *

# construct the url

base_url = "https://docs.google.com/spreadsheets/d/"
spreadsheet_key = "1g1wE5-e_B3VQw6X2-yQTtw4NkUpc-bbk5x_lUBRkR5A"
export_url = "/export?output=xlsx"

url = base_url + spreadsheet_key + export_url

# load spreadsheet

heliconia = pandas.read_excel(url)
print heliconia.head()

  species     pop_and_year morph  plant_id  corolla_length  mean_bracts  \
0   Bihai  Boeri Lake 2008   NaN         1       48.205000            4   
1   Bihai  Boeri Lake 2008   NaN         2       49.700000            5   
2   Bihai  Boeri Lake 2008   NaN         3       48.123333            3   
3   Bihai  Boeri Lake 2008   NaN         4       48.060000            4   
4   Bihai  Boeri Lake 2008   NaN         5       47.240000            4   

   seeds_per_plant  
0        51.176471  
1        28.411765  
2        39.111111  
3        15.108108  
4        32.645161  


You can load a specific worksheet by including an integer (0, 1, ...) as the second argument. First sheet is index 0.

In [45]:
# load the populations sheet (3rd sheet)

populations = pandas.read_excel(url, 2)
print populations.head()

           pop_and_year       population  year
0       Boeri Lake 2008       Boeri Lake  2008
1  Freshwater Lake 2008  Freshwater Lake  2008
2       Boeri Lake 2009       Boeri Lake  2009
3  Freshwater Lake 2009  Freshwater Lake  2009
4        Syndicate 2008        Syndicate  2008


You can also specify a worksheet by passing its name as a string instead.

In [46]:
# load the populations sheet (same as above)

populations = pandas.read_excel(url, "populations")
print populations.head()

           pop_and_year       population  year
0       Boeri Lake 2008       Boeri Lake  2008
1  Freshwater Lake 2008  Freshwater Lake  2008
2       Boeri Lake 2009       Boeri Lake  2009
3  Freshwater Lake 2009  Freshwater Lake  2009
4        Syndicate 2008        Syndicate  2008


## Joining data

When normalizing the data set, we noticed that the attributes `population` and `year` were recorded as a combined variable `pop_and_year`. This means that we cannot group the data *only* by `population` or *only* by `year`. But these would be reasonable comparisons for an analyst to make!

We *did* create a lookup table (`populations`) that translates each `pop_and_year` to `population` and `year`. We can have pandas apply this translation to each row in the data set. 

Among Python and R users, this operation is called **merging**. (When researching information about data management, however, you will often see the term **joining** instead, so be ready to recognize this term.) To `merge()` two dataframes, pass the name of each as the first two parameters, and then specify the common variable (e.g., `pop_and_year`) for the `on` parameter.

In [47]:
# merge the two tables heliconia and populations

joined = pandas.merge(heliconia, populations, on = 'pop_and_year')
joined.head()

,species,pop_and_year,morph,plant_id,corolla_length,mean_bracts,seeds_per_plant,population,year
0,Bihai,Boeri Lake 2008,NaN,1,48.205000,4,51.176471,Boeri Lake,2008
1,Bihai,Boeri Lake 2008,NaN,2,49.700000,5,28.411765,Boeri Lake,2008
2,Bihai,Boeri Lake 2008,NaN,3,48.123333,3,39.111111,Boeri Lake,2008
3,Bihai,Boeri Lake 2008,NaN,4,48.060000,4,15.108108,Boeri Lake,2008
4,Bihai,Boeri Lake 2008,NaN,5,47.240000,4,32.645161,Boeri Lake,2008


Notice that `joined` has columns from both `heliconia` and `populations`; and that `population` and `year` data appear as separate variables.

The pandas documentation has some [excellent illustrations of the concept of merging](http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging).